In [1]:
import os
import errno
import argparse
import sys
import pickle

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

from data_utils import load_CIFAR_data, generate_partial_data, generate_bal_private_data
from FedMD import FedMD
from Neural_Networks import train_models, cnn_2layer_fc_model, cnn_3layer_fc_model

import numpy as np
from tensorflow.keras.models import clone_model, load_model
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from data_utils import generate_alignment_data
from Neural_Networks import remove_last_layer

## Functions and Configurations

In [2]:


CANDIDATE_MODELS = {"2_layer_CNN": cnn_2layer_fc_model, 
                    "3_layer_CNN": cnn_3layer_fc_model} 

conf_file = os.path.abspath("conf/CIFAR_balance_conf.json")

with open(conf_file, "r") as f:
    conf_dict = eval(f.read())
    
    #n_classes = conf_dict["n_classes"]
    model_config = conf_dict["models"]
    pre_train_params = conf_dict["pre_train_params"]
    model_saved_dir = conf_dict["model_saved_dir"]
    model_saved_names = conf_dict["model_saved_names"]
    is_early_stopping = conf_dict["early_stopping"]
    public_classes = conf_dict["public_classes"]
    private_classes = conf_dict["private_classes"]
    n_classes = len(public_classes) + len(private_classes)
    
    emnist_data_dir = conf_dict["EMNIST_dir"]    
    N_parties = conf_dict["N_parties"]
    N_samples_per_class = conf_dict["N_samples_per_class"]
    
    N_rounds = conf_dict["N_rounds"]
    N_alignment = conf_dict["N_alignment"]
    N_private_training_round = conf_dict["N_private_training_round"]
    private_training_batchsize = conf_dict["private_training_batchsize"]
    N_logits_matching_round = conf_dict["N_logits_matching_round"]
    logits_matching_batchsize = conf_dict["logits_matching_batchsize"]
    aug = conf_dict["aug"]
    compress = conf_dict["compress"]
    
    
    result_save_dir = conf_dict["result_save_dir"]

del conf_dict, conf_file

## Play 


In [3]:


X_train_CIFAR10, y_train_CIFAR10, X_test_CIFAR10, y_test_CIFAR10 \
= load_CIFAR_data(data_type="CIFAR10", 
                    standarized = True, verbose = True)

public_dataset = {"X": X_train_CIFAR10, "y": y_train_CIFAR10}


X_train_CIFAR100, y_train_CIFAR100, X_test_CIFAR100, y_test_CIFAR100 \
= load_CIFAR_data(data_type="CIFAR100",
                    standarized = True, verbose = True)

# only use those CIFAR100 data whose y_labels belong to private_classes
X_train_CIFAR100, y_train_CIFAR100 \
= generate_partial_data(X = X_train_CIFAR100, y= y_train_CIFAR100,
                        class_in_use = private_classes, 
                        verbose = True)


X_test_CIFAR100, y_test_CIFAR100 \
= generate_partial_data(X = X_test_CIFAR100, y= y_test_CIFAR100,
                        class_in_use = private_classes, 
                        verbose = True)

# relabel the selected CIFAR100 data for future convenience
for index, cls_ in enumerate(private_classes):        
    y_train_CIFAR100[y_train_CIFAR100 == cls_] = index + len(public_classes)
    y_test_CIFAR100[y_test_CIFAR100 == cls_] = index + len(public_classes)
del index, cls_

print(pd.Series(y_train_CIFAR100).value_counts())
mod_private_classes = np.arange(len(private_classes)) + len(public_classes)

print("="*60)
#generate private data
private_data, total_private_data\
=generate_bal_private_data(X_train_CIFAR100, y_train_CIFAR100,      
                            N_parties = N_parties,           
                            classes_in_use = mod_private_classes, 
                            N_samples_per_class = N_samples_per_class, 
                            data_overlap = False)

print("="*60)
X_tmp, y_tmp = generate_partial_data(X = X_test_CIFAR100, y= y_test_CIFAR100,
                                        class_in_use = mod_private_classes, 
                                        verbose = True)
private_test_data = {"X": X_tmp, "y": y_tmp}
del X_tmp, y_tmp

parties = []
if model_saved_dir is None:
    for i, item in enumerate(model_config):
        model_name = item["model_type"]
        model_params = item["params"]
        tmp = CANDIDATE_MODELS[model_name](n_classes=n_classes, 
                                            input_shape=(32,32,3),
                                            **model_params)
        print("model {0} : {1}".format(i, model_saved_names[i]))
        # print(tmp.summary())
        parties.append(tmp)


X_train shape : (50000, 32, 32, 3)
X_test shape : (10000, 32, 32, 3)
y_train shape : (50000,)
y_test shape : (10000,)
X_train shape : (50000, 32, 32, 3)
X_test shape : (10000, 32, 32, 3)
y_train shape : (50000,)
y_test shape : (10000,)
X shape : (3000, 32, 32, 3)
y shape : (3000,)
X shape : (600, 32, 32, 3)
y shape : (600,)
10    500
15    500
14    500
12    500
13    500
11    500
dtype: int64
X shape : (600, 32, 32, 3)
y shape : (600,)
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-11 16:50:09.219360: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-11 16:50:09.220336: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/gadmohamed/miniforge3/envs/env_tf/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


model 0 : CNN_128_256
model 1 : CNN_128_384
model 2 : CNN_128_512
model 3 : CNN_256_256
model 4 : CNN_256_512
model 5 : CNN_64_128_256
model 6 : CNN_64_128_192
model 7 : CNN_128_192_256
model 8 : CNN_128_128_128
model 9 : CNN_128_128_192
model 10 : CNN_128_256
model 11 : CNN_128_384
model 12 : CNN_128_512
model 13 : CNN_256_256
model 14 : CNN_256_512
model 15 : CNN_64_128_256
model 16 : CNN_64_128_192
model 17 : CNN_128_192_256
model 18 : CNN_128_128_128
model 19 : CNN_128_128_192


In [4]:
private_data[0]['X'].shape, total_private_data['X'].shape

((60, 32, 32, 3), (1200, 32, 32, 3))

In [5]:

central_trials = [] 
for i, item in enumerate(model_config):
    model_name = item["model_type"]
    model_params = item["params"]
    trials = [] 
    for trail in range(3) : 
        tf.keras.backend.clear_session()
        model_ub = CANDIDATE_MODELS[model_name](n_classes=n_classes, 
                                            input_shape=(32,32,3),
                                            **model_params)
        model_ub.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-6),
                            loss = "sparse_categorical_crossentropy", 
                            metrics = ["accuracy"])

        ub_history = model_ub.fit(total_private_data["X"], total_private_data["y"],
                        batch_size = 32, epochs = 50, shuffle=True, verbose = True, 
                        validation_data = [private_test_data["X"], private_test_data["y"]],
                        callbacks=[EarlyStopping(monitor="val_accuracy", min_delta=0.001, patience=4, restore_best_weights=True)])
        trials.append(ub_history.history["val_accuracy"][-1])
    avg_trial = np.mean(trials)
    central_trials.append(avg_trial)
    print("trials for model {0}: {1}".format(i, avg_trial))


Epoch 1/50


2023-05-11 16:50:10.956318: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-11 16:50:11.347627: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - ETA: 0s - loss: 2.8714 - accuracy: 0.0958

2023-05-11 16:50:12.791092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 2s 36ms/step - loss: 2.8714 - accuracy: 0.0958 - val_loss: 2.7440 - val_accuracy: 0.1200
Epoch 2/50
38/38 [==============================] - 1s 22ms/step - loss: 2.6799 - accuracy: 0.1383 - val_loss: 2.6817 - val_accuracy: 0.1267
Epoch 3/50
38/38 [==============================] - 1s 28ms/step - loss: 2.5289 - accuracy: 0.1617 - val_loss: 2.6074 - val_accuracy: 0.1500
Epoch 4/50
38/38 [==============================] - 1s 25ms/step - loss: 2.4266 - accuracy: 0.1867 - val_loss: 2.5249 - val_accuracy: 0.1833
Epoch 5/50
38/38 [==============================] - 1s 22ms/step - loss: 2.2976 - accuracy: 0.2008 - val_loss: 2.4373 - val_accuracy: 0.2017
Epoch 6/50
38/38 [==============================] - 1s 22ms/step - loss: 2.1920 - accuracy: 0.2333 - val_loss: 2.3466 - val_accuracy: 0.2117
Epoch 7/50
38/38 [==============================] - 1s 23ms/step - loss: 2.1201 - accuracy: 0.2458 - val_loss: 2.2528 - val_accuracy: 0.2550
Epoch 8/50
38/38 [======

...

38/38 [==============================] - 4s 60ms/step - loss: 3.1685 - accuracy: 0.0933 - val_loss: 2.7946 - val_accuracy: 0.1317
Epoch 2/50
38/38 [==============================] - 1s 30ms/step - loss: 3.1307 - accuracy: 0.1033 - val_loss: 2.7993 - val_accuracy: 0.1433
Epoch 3/50
38/38 [==============================] - 1s 26ms/step - loss: 3.0736 - accuracy: 0.1108 - val_loss: 2.8020 - val_accuracy: 0.1517
Epoch 4/50
38/38 [==============================] - 2s 41ms/step - loss: 3.0615 - accuracy: 0.1042 - val_loss: 2.7996 - val_accuracy: 0.1600
Epoch 5/50
38/38 [==============================] - 2s 41ms/step - loss: 3.0000 - accuracy: 0.1167 - val_loss: 2.7935 - val_accuracy: 0.1617
Epoch 6/50
38/38 [==============================] - 1s 23ms/step - loss: 2.9767 - accuracy: 0.1133 - val_loss: 2.7849 - val_accuracy: 0.1633
Epoch 7/50
38/38 [==============================] - 1s 23ms/step - loss: 2.9380 - accuracy: 0.1275 - val_loss: 2.7725 - val_accuracy: 0.1700
Epoch 8/50
38/38 [======

2023-05-11 19:39:30.200594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - ETA: 0s - loss: 3.1700 - accuracy: 0.0867

2023-05-11 19:39:32.604273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 3s 62ms/step - loss: 3.1700 - accuracy: 0.0867 - val_loss: 2.7806 - val_accuracy: 0.0733
Epoch 2/50
38/38 [==============================] - 1s 34ms/step - loss: 3.1041 - accuracy: 0.0950 - val_loss: 2.7707 - val_accuracy: 0.0683
Epoch 3/50
38/38 [==============================] - 1s 33ms/step - loss: 3.0697 - accuracy: 0.1025 - val_loss: 2.7627 - val_accuracy: 0.0833
Epoch 4/50
38/38 [==============================] - 1s 24ms/step - loss: 3.0269 - accuracy: 0.1183 - val_loss: 2.7562 - val_accuracy: 0.1017
Epoch 5/50
38/38 [==============================] - 1s 26ms/step - loss: 3.0185 - accuracy: 0.1150 - val_loss: 2.7483 - val_accuracy: 0.1133
Epoch 6/50
38/38 [==============================] - 1s 23ms/step - loss: 2.9569 - accuracy: 0.1183 - val_loss: 2.7358 - val_accuracy: 0.1200
Epoch 7/50
38/38 [==============================] - 1s 26ms/step - loss: 2.9435 - accuracy: 0.1333 - val_loss: 2.7172 - val_accuracy: 0.1217
Epoch 8/50
38/38 [======

In [6]:

local_trials = []
for i, item in enumerate(model_config):
    model_name = item["model_type"]
    model_params = item["params"]
    trials = [] 
    for trail in range(3) : 
        tf.keras.backend.clear_session()
        model_l = CANDIDATE_MODELS[model_name](n_classes=n_classes, 
                                            input_shape=(32,32,3),
                                            **model_params)
        model_l.compile(optimizer=tf.keras.optimizers.Adam(lr = 1e-6),
                            loss = "sparse_categorical_crossentropy", 
                            metrics = ["accuracy"])

        ub_history = model_l.fit(private_data[i]["X"], private_data[i]["y"],
                        batch_size = 32, epochs = 50, shuffle=True, verbose = True, 
                        validation_data = [private_test_data["X"], private_test_data["y"]],
                        callbacks=[EarlyStopping(monitor="val_accuracy", min_delta=0.001, patience=4, restore_best_weights=True)])
        trials.append(ub_history.history["val_accuracy"][-1])
    avg_trial = np.mean(trials)
    local_trials.append(avg_trial)
    print("trials for model {0}: {1}".format(i, avg_trial))



Epoch 1/50


2023-05-11 19:40:23.709969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 3.2388 - accuracy: 0.0667

2023-05-11 19:40:24.690449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 2s 911ms/step - loss: 3.2388 - accuracy: 0.0667 - val_loss: 2.8040 - val_accuracy: 0.0450
Epoch 2/50
2/2 [==============================] - 0s 232ms/step - loss: 3.0775 - accuracy: 0.0333 - val_loss: 2.8053 - val_accuracy: 0.0417
Epoch 3/50
2/2 [==============================] - 0s 238ms/step - loss: 3.1514 - accuracy: 0.0667 - val_loss: 2.8065 - val_accuracy: 0.0350
Epoch 4/50
2/2 [==============================] - 0s 185ms/step - loss: 3.0655 - accuracy: 0.1000 - val_loss: 2.8075 - val_accuracy: 0.0300
Epoch 5/50
2/2 [==============================] - 0s 241ms/step - loss: 3.0670 - accuracy: 0.1167 - val_loss: 2.8083 - val_accuracy: 0.0267
Epoch 1/50


...

2/2 [==============================] - 14s 11s/step - loss: 3.4455 - accuracy: 0.0167 - val_loss: 2.8098 - val_accuracy: 0.0017
Epoch 2/50
2/2 [==============================] - 3s 2s/step - loss: 3.5547 - accuracy: 0.0500 - val_loss: 2.8093 - val_accuracy: 0.0117
Epoch 3/50
2/2 [==============================] - 1s 693ms/step - loss: 3.4988 - accuracy: 0.0333 - val_loss: 2.8102 - val_accuracy: 0.0383
Epoch 4/50
2/2 [==============================] - 1s 298ms/step - loss: 3.4612 - accuracy: 0.0167 - val_loss: 2.8125 - val_accuracy: 0.0400
Epoch 5/50
2/2 [==============================] - 0s 286ms/step - loss: 3.4298 - accuracy: 0.0667 - val_loss: 2.8154 - val_accuracy: 0.0333
Epoch 6/50
2/2 [==============================] - 0s 213ms/step - loss: 3.4423 - accuracy: 0.0500 - val_loss: 2.8186 - val_accuracy: 0.0283
Epoch 7/50
2/2 [==============================] - 0s 268ms/step - loss: 3.5339 - accuracy: 0.0333 - val_loss: 2.8217 - val_accuracy: 0.0217
Epoch 8/50
2/2 [===================

2023-05-11 19:50:14.431766: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 3.5327 - accuracy: 0.0500

2023-05-11 19:50:17.904748: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 6s 4s/step - loss: 3.5327 - accuracy: 0.0500 - val_loss: 2.8028 - val_accuracy: 0.1217
Epoch 2/50
2/2 [==============================] - 2s 634ms/step - loss: 3.4960 - accuracy: 0.0333 - val_loss: 2.8024 - val_accuracy: 0.0917
Epoch 3/50
2/2 [==============================] - 1s 443ms/step - loss: 3.5073 - accuracy: 0.0167 - val_loss: 2.8009 - val_accuracy: 0.0800
Epoch 4/50
2/2 [==============================] - 0s 446ms/step - loss: 3.4362 - accuracy: 0.0667 - val_loss: 2.7987 - val_accuracy: 0.0733
Epoch 5/50
2/2 [==============================] - 0s 265ms/step - loss: 3.5371 - accuracy: 0.0167 - val_loss: 2.7964 - val_accuracy: 0.0833
trials for model 19: 0.052222223952412605


In [7]:
local_trials

[0.07555555490156014,
 0.05000000074505806,
 0.14111111561457315,
 0.11388888955116272,
 0.09888888895511627,
 0.0716666653752327,
 0.10333333661158879,
 0.06833333459993203,
 0.11111111318071683,
 0.025555555941537023,
 0.23888888955116272,
 0.17555556197961172,
 0.10888889307777087,
 0.07777778121332328,
 0.08055555758376916,
 0.09444444564481576,
 0.05722222228844961,
 0.07055555641030271,
 0.05722222135712703,
 0.052222223952412605]

In [8]:
central_trials

[0.67166668176651,
 0.7333333492279053,
 0.7361111243565878,
 0.646666685740153,
 0.6766666769981384,
 0.4750000114242236,
 0.22222223039716482,
 0.44111111760139465,
 0.02944444554547469,
 0.21222222596406937,
 0.7777778108914694,
 0.7211111386617025,
 0.7322222391764323,
 0.6516667008399963,
 0.6744444767634074,
 0.2700000007947286,
 0.43222224712371826,
 0.2000000129143397,
 0.4144444515307744,
 0.38722223540147144]